In [60]:
# from utils import *
from dotenv import load_dotenv, find_dotenv 
load_dotenv()

import pandas as pd
import polars as pl


In [61]:

metrics = (pl.from_pandas(pd.read_excel('financialtools/data/metrics.xlsx'))
    .filter(pl.col("ticker") == "CPR.MI")
    .to_pandas())

metrics

,ticker,time,GrossMargin,OperatingMargin,NetProfitMargin,EBITDAMargin,ROA,ROE,FCFToRevenue,FCFYield,FCFtoDebt,DebtToEquity,CurrentRatio
0,CPR.MI,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CPR.MI,2021,0.596861,0.184471,0.131081,0.228564,0.055919,0.120078,0.155521,0.094584,0.220461,0.646218,0.355148
2,CPR.MI,2022,0.588857,0.204070,0.123443,0.221308,0.055359,0.124430,0.009305,0.005846,0.014028,0.668597,0.278036
3,CPR.MI,2023,0.582505,0.207325,0.113239,0.221853,0.049509,0.112984,-0.054410,-0.033986,-0.070456,0.770511,0.318422
4,CPR.MI,2024,0.575542,0.193921,0.065672,0.160727,0.023764,0.052309,0.068539,0.033131,0.073474,0.743020,0.366508


In [62]:
metrics = metrics.to_json(orient="records") 

In [63]:

composite_scores = (pl.from_pandas(pd.read_excel('financialtools/data/composite_scores.xlsx'))
    .filter(pl.col("ticker") == "CPR.MI")
    .to_pandas())

composite_scores = composite_scores.to_json(orient="records") 

composite_scores


'[{"ticker":"CPR.MI","time":2020,"composite_score":3.0},{"ticker":"CPR.MI","time":2021,"composite_score":3.52},{"ticker":"CPR.MI","time":2022,"composite_score":3.04},{"ticker":"CPR.MI","time":2023,"composite_score":2.86},{"ticker":"CPR.MI","time":2024,"composite_score":2.74}]'

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict
from langchain_core.output_parsers import PydanticOutputParser
from langchain.output_parsers import OutputFixingParser

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Pydantic output model
class StockRegimeAssessment(BaseModel):
    regime: Literal["bull", "bear", "postpone"] = Field(
        ..., description="The fundamental regime classification of the stock"
    )
    rationale: str = Field(
        ..., description="Concise explanation justifying the regime classification"
    )
    metrics_movement: Dict[str, str] = Field(
            ..., description=(
                "Summary of how key financial metrics have moved across years. "
                "Keys are metric names, values describe the movement, e.g., "
                "'GrossMargin': 'increased steadily', 'DebtToEquity': 'rose sharply'"
            )
        )


# Instantiate the LLM (OpenAI GPT-4 or your preferred model)
llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

# Instantiate the parser with the Pydantic model
parser = PydanticOutputParser(pydantic_object=StockRegimeAssessment)
# Wrap your parser with OutputFixingParser
parser = OutputFixingParser.from_llm(parser=parser, llm=llm)

# Get the format instructions string from the parser
format_instructions = parser.get_format_instructions()

system_prompt_template = """
You are a trader assistant specializing in fundamental analysis. 

Based on the following financial data, provide a concise overall assessment that classifies 
the stock’s current fundamental regime as one of:

- bull: Strong and improving fundamentals supporting a positive outlook.
- bear: Weak or deteriorating fundamentals indicating risk or decline.
- postpone: Mixed or inconclusive fundamentals suggesting to wait for clearer signals.

Financial data constists of financial metrics, composite score and red flags.

Profitability and Margin Metrics:
    -GrossMargin: gross profit / total revenue 
    -OperatingMargin: operating income / total revenue
    -NetProfitMargin: net income / total revenue
    -EBITDAMargin: ebitda / total revenue
Returns metrics:
    -ROA: net income / total assets
    -ROE: net income / total equity
Cash Flow Strength metrics: 
    -FCFToRevenue: free cash flow / total revenue
    -FCFYield: free cash flow / market capitalization
    -FCFToDebt:: free cash flow / total debt
Leverage & Solvency metrics:
    -DebtToEquity: total debt / total equity
Liquidity metrics:
    -CurrentRatio: working capital / total liabilities


"""

# Create a ChatPromptTemplate with system message and user input

# Format the system prompt with format_instructions
system_prompt_filled = system_prompt_template.format(format_instructions=format_instructions)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt_filled),
    ("human", "{financial_data}"),
])




# Create a runnable chain: prompt followed by LLM invocation
chain = prompt | llm | parser

# Then invoke with a dict containing 'financial_data'
response = chain.invoke({"financial_data": metrics})

In [45]:
    response

StockRegimeAssessment(regime='postpone', rationale="The company's fundamentals are mixed, showing some stability in margins and efficiency metrics but also signs of weakening profitability and liquidity issues, such as declining net profit margin, negative cash flow in 2023, and low current ratio. These conflicting signals justify a cautious wait-and-see approach.", metrics_movement={'GrossMargin': 'remained stable around 58-59%', 'OperatingMargin': 'remained stable around 18-20%', 'ROE': 'modest but positive', 'ROA': 'modest but positive', 'FreeCashFlowToRevenue': 'positive in 2024', 'FCF_Yield': 'low (~3%)', 'FCF_to_Debt': 'modest (~7%)', 'NetProfitMargin': 'declined from ~13% in 2021-2022 to ~6.5% in 2024', 'CurrentRatio': 'below 1 (~0.28-0.37)', 'DebtToEquity': 'high (~0.74-0.77)', 'FreeCashFlow': 'negative in 2023'})

In [ ]:
# 7. Prepare batch inputs by converting each dict to string
# batch_inputs = [{"input": str(stock_input)} for stock_input in json_data]

In [51]:
prompt_value = prompt.invoke({"financial_data": metrics})

prompt_value

ChatPromptValue(messages=[SystemMessage(content='\nYou are a trader assistant specializing in fundamental analysis. \n\nBased on the following financial data, provide a concise overall assessment that classifies \nthe stock’s current fundamental regime as one of:\n\n- bull: Strong and improving fundamentals supporting a positive outlook.\n- bear: Weak or deteriorating fundamentals indicating risk or decline.\n- postpone: Mixed or inconclusive fundamentals suggesting to wait for clearer signals.\n\nYour assessment should:\n- Summarize key strengths and weaknesses.\n- Highlight significant red flags impacting the outlook.\n- Justify the regime classification clearly and succinctly.\n\nMetrics the user will input:\n    -GrossMargin: gross profit / total revenue \n    -OperatingMargin: operating income / total revenue\n    -NetProfitMargin: net income / total revenue\n    -EBITDAMargin: ebitda / total revenue\n    -ROA: net income / total assets\n    -ROE: net income / total equity\n    -F

In [52]:
prompt_value.messages

[SystemMessage(content='\nYou are a trader assistant specializing in fundamental analysis. \n\nBased on the following financial data, provide a concise overall assessment that classifies \nthe stock’s current fundamental regime as one of:\n\n- bull: Strong and improving fundamentals supporting a positive outlook.\n- bear: Weak or deteriorating fundamentals indicating risk or decline.\n- postpone: Mixed or inconclusive fundamentals suggesting to wait for clearer signals.\n\nYour assessment should:\n- Summarize key strengths and weaknesses.\n- Highlight significant red flags impacting the outlook.\n- Justify the regime classification clearly and succinctly.\n\nMetrics the user will input:\n    -GrossMargin: gross profit / total revenue \n    -OperatingMargin: operating income / total revenue\n    -NetProfitMargin: net income / total revenue\n    -EBITDAMargin: ebitda / total revenue\n    -ROA: net income / total assets\n    -ROE: net income / total equity\n    -FCFToRevenue: free cash fl

In [56]:
print(prompt_value.messages[0].content)


You are a trader assistant specializing in fundamental analysis. 

Based on the following financial data, provide a concise overall assessment that classifies 
the stock’s current fundamental regime as one of:

- bull: Strong and improving fundamentals supporting a positive outlook.
- bear: Weak or deteriorating fundamentals indicating risk or decline.
- postpone: Mixed or inconclusive fundamentals suggesting to wait for clearer signals.

Your assessment should:
- Summarize key strengths and weaknesses.
- Highlight significant red flags impacting the outlook.
- Justify the regime classification clearly and succinctly.

Metrics the user will input:
    -GrossMargin: gross profit / total revenue 
    -OperatingMargin: operating income / total revenue
    -NetProfitMargin: net income / total revenue
    -EBITDAMargin: ebitda / total revenue
    -ROA: net income / total assets
    -ROE: net income / total equity
    -FCFToRevenue: free cash flow / total revenue
    -FCFYield: free cash fl

In [57]:
print(prompt_value.messages[1].content)

   ticker  time  GrossMargin  OperatingMargin  NetProfitMargin  EBITDAMargin  \
0  CPR.MI  2020          NaN              NaN              NaN           NaN   
1  CPR.MI  2021     0.596861         0.184471         0.131081      0.228564   
2  CPR.MI  2022     0.588857         0.204070         0.123443      0.221308   
3  CPR.MI  2023     0.582505         0.207325         0.113239      0.221853   
4  CPR.MI  2024     0.575542         0.193921         0.065672      0.160727   

        ROA       ROE  FCFToRevenue  FCFYield  FCFtoDebt  DebtToEquity  \
0       NaN       NaN           NaN       NaN        NaN           NaN   
1  0.055919  0.120078      0.155521  0.094584   0.220461      0.646218   
2  0.055359  0.124430      0.009305  0.005846   0.014028      0.668597   
3  0.049509  0.112984     -0.054410 -0.033986  -0.070456      0.770511   
4  0.023764  0.052309      0.068539  0.033131   0.073474      0.743020   

   CurrentRatio  
0           NaN  
1      0.355148  
2      0.278036  
3 

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict
from langchain_core.output_parsers import PydanticOutputParser
from langchain.output_parsers import OutputFixingParser

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Pydantic output model
class StockRegimeAssessment(BaseModel):
    regime: Literal["bull", "bear", "postpone"] = Field(
        ..., description="The fundamental regime classification of the stock"
    )
    rationale: str = Field(
        ..., description="Concise explanation justifying the regime classification"
    )
    metrics_movement: Dict[str, str] = Field(
            ..., description=(
                "Summary of how key financial metrics have moved across years. "
                "Keys are metric names, values describe the movement, e.g., "
                "'GrossMargin': 'increased steadily', 'DebtToEquity': 'rose sharply'"
            )
        )


# Instantiate the LLM (OpenAI GPT-4 or your preferred model)
llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

# Instantiate the parser with the Pydantic model
parser = PydanticOutputParser(pydantic_object=StockRegimeAssessment)
# Wrap your parser with OutputFixingParser
parser = OutputFixingParser.from_llm(parser=parser, llm=llm)

# Get the format instructions string from the parser
format_instructions = parser.get_format_instructions()

system_prompt_template = """
You are a trader assistant specializing in fundamental analysis. 

Based on the following financial data, provide a concise overall assessment that classifies 
the stock’s current fundamental regime as one of:

- bull: Strong and improving fundamentals supporting a positive outlook.
- bear: Weak or deteriorating fundamentals indicating risk or decline.
- postpone: Mixed or inconclusive fundamentals suggesting to wait for clearer signals.

Financial data constists of financial metrics, composite score and red flags.

Profitability and Margin Metrics:
    -GrossMargin: gross profit / total revenue 
    -OperatingMargin: operating income / total revenue
    -NetProfitMargin: net income / total revenue
    -EBITDAMargin: ebitda / total revenue
Returns metrics:
    -ROA: net income / total assets
    -ROE: net income / total equity
Cash Flow Strength metrics: 
    -FCFToRevenue: free cash flow / total revenue
    -FCFYield: free cash flow / market capitalization
    -FCFToDebt:: free cash flow / total debt
Leverage & Solvency metrics:
    -DebtToEquity: total debt / total equity
Liquidity metrics:
    -CurrentRatio: working capital / total liabilities

The composite score is a weighted average (1 to 5) that summarizes the company’s overall fundamental health.
It reflects profitability, efficiency, leverage, liquidity, and cash flow strength, based on the above mentioned financial metrics.

Range:
1 = Weak fundamentals
5 = Strong fundamentals

Each metric is scored on a 1–5 scale and multiplied by its weight. The composite score is the sum of weighted scores divided by the total weight.



"""

# Create a ChatPromptTemplate with system message and user input

system_prompt_filled = system_prompt_template.format(format_instructions=format_instructions)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt_filled),
    ("human", "Metrics:\n{metrics}\nScores:\n{scores}"),
])




# Create a runnable chain: prompt followed by LLM invocation
chain = prompt | llm | parser



In [ ]:
import json

metrics_str = json.dumps(metrics, indent=2)
scores_str = json.dumps(composite_scores, indent=2)

# Then invoke with a dict containing 'financial_data'
response = chain.invoke({
    "metrics": metrics_str,  
    "scores": scores_str,    
})

response


StockRegimeAssessment(regime='postpone', rationale="The company's fundamental score has been gradually declining from 2021 through 2024, indicating weakening financial health. Key metrics such as profit margins, cash flow, and leverage ratios show deterioration, especially in free cash flow and debt levels. The declining composite score suggests that the company's fundamentals are weakening, and current indicators do not support a positive outlook.", metrics_movement={'ProfitMargin': 'decreased steadily', 'CashFlow': 'declined over the period', 'DebtToEquity': 'rose sharply', 'FreeCashFlow': 'deteriorated significantly'})

In [79]:
red_flags = pl.concat([
    (pl.from_pandas(pd.read_excel('financialtools/data/red_flags.xlsx'))
        .filter(pl.col("ticker") == "CPR.MI")),
    (pl.from_pandas(pd.read_excel('financialtools/data/raw_red_flags.xlsx'))
        .filter(pl.col("ticker") == "CPR.MI"))]
).to_pandas()

red_flags = red_flags.to_json(orient="records") 

red_flags

'[{"ticker":"CPR.MI","time":2022,"metrics":"FCFtoDebt","red_flag":"Insufficient Free Cash Flow to cover debt"},{"ticker":"CPR.MI","time":2023,"metrics":"FCFtoDebt","red_flag":"Insufficient Free Cash Flow to cover debt"},{"ticker":"CPR.MI","time":2023,"metrics":"rrf_fcf","red_flag":"Negative Free Cash Flow"},{"ticker":"CPR.MI","time":2023,"metrics":"ebitdaVSocf","red_flag":"Earnings quality concern (EBITDA >> OCF)"}]'

In [82]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict, Optional
from langchain_core.output_parsers import PydanticOutputParser
from langchain.output_parsers import OutputFixingParser

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Pydantic output model
class StockRegimeAssessment(BaseModel):
    regime: Literal["bull", "bear", "postpone"] = Field(
        ..., description="The fundamental regime classification of the stock"
    )
    rationale: str = Field(
        ..., description="Concise explanation justifying the regime classification based on the financial metrics, composite ratio and red flags"
    )
    metrics_movement: str = Field(
        ..., description=(
            "A summary description of how key financial metrics have moved across years, "
            "e.g., 'GrossMargin increased steadily, DebtToEquity rose sharply, FCFYield remained stable.'"
        )
    )
    non_aligned_findings: Optional[str] = Field(
        None,
        description=(
            "Observations or signals that are not aligned with the overall metric trends, "
            "such as contradictory indicators, anomalies, or important red flags."
        )
    )


# Instantiate the LLM (OpenAI GPT-4 or your preferred model)
llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

# Instantiate the parser with the Pydantic model
parser = PydanticOutputParser(pydantic_object=StockRegimeAssessment)
# Wrap your parser with OutputFixingParser
parser = OutputFixingParser.from_llm(parser=parser, llm=llm)

# Get the format instructions string from the parser
format_instructions = parser.get_format_instructions()

system_prompt_template = """
You are a trader assistant specializing in fundamental analysis. 

Based on the following financial data, provide a concise overall assessment that classifies 
the stock’s current fundamental regime as one of:

- bull: Strong and improving fundamentals supporting a positive outlook.
- bear: Weak or deteriorating fundamentals indicating risk or decline.
- postpone: Mixed or inconclusive fundamentals suggesting to wait for clearer signals.

Financial data constists of financial metrics, composite score and red flags.

Profitability and Margin Metrics:
    -GrossMargin: gross profit / total revenue 
    -OperatingMargin: operating income / total revenue
    -NetProfitMargin: net income / total revenue
    -EBITDAMargin: ebitda / total revenue
Returns metrics:
    -ROA: net income / total assets
    -ROE: net income / total equity
Cash Flow Strength metrics: 
    -FCFToRevenue: free cash flow / total revenue
    -FCFYield: free cash flow / market capitalization
    -FCFToDebt:: free cash flow / total debt
Leverage & Solvency metrics:
    -DebtToEquity: total debt / total equity
Liquidity metrics:
    -CurrentRatio: working capital / total liabilities

The composite score is a weighted average (1 to 5) that summarizes the company’s overall fundamental health.
It reflects profitability, efficiency, leverage, liquidity, and cash flow strength, based on the above mentioned financial metrics.

Range:
1 = Weak fundamentals
5 = Strong fundamentals

Each metric is scored on a 1–5 scale and multiplied by its weight. The composite score is the sum of weighted scores divided by the total weight.

A red flag is an early warning signal that highlights potential weaknesses in a company’s financial statements 
or business quality. These warnings do not always mean immediate distress, but they indicate heightened risk that 
traders should carefully consider before taking a position.

"""

# Create a ChatPromptTemplate with system message and user input

system_prompt_filled = system_prompt_template.format(format_instructions=format_instructions)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt_filled),
    ("human", "Metrics:\n{metrics}\nScores:\n{scores}\nRedFlags:\n{red_flags}"),
])

# Create a runnable chain: prompt followed by LLM invocation
chain = prompt | llm | parser

red_flags_str = json.dumps(red_flags, indent=2)

# Then invoke with a dict containing 'financial_data'
response = chain.invoke({
    "metrics": metrics_str,  
    "scores": scores_str,
    "red_flags": red_flags_str,    
})

response

StockRegimeAssessment(regime='postpone', rationale="The company's declining fundamental score, persistent negative free cash flow, and rising debt levels indicate financial stress and increased risk. These red flags outweigh the modest profitability metrics, suggesting that the company is not currently in a stable or improving financial position.", metrics_movement='Fundamental score declined from 3.0 in 2020 to 2.74 in 2024, free cash flow remained negative, and debt levels increased, signaling deteriorating financial health.', non_aligned_findings=None)

In [28]:
batch_inputs

[{'input': 'ticker'},
 {'input': 'date'},
 {'input': 'rclose'},
 {'input': 'ema_st'},
 {'input': 'ema_mt'},
 {'input': 'ema_lt'},
 {'input': 'spread_stmt'},
 {'input': 'spread_mtlt'},
 {'input': 'spread_stlt'},
 {'input': 'rrg'},
 {'input': 'RSI'}]

In [20]:


# 8. Run batch in parallel
results = chain.batch(batch_inputs)